#Data Analytics on Dynamic Pricing Urban Parking

This notebook demonstrates how to process live data streams using Pathway. The dataset used here is a subset of the one provided — specifically, it includes data for only a single parking spot. You are expected to implement your model across all parking spots.

In [ ]:
!pip install pathway bokeh --quiet # This cell may take a few seconds to execute.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

# Step 1: Importing and Preprocessing the Data

In [ ]:
df = pd.read_csv('/content/dataset.csv')
df

# You can find the sample dataset here: https://drive.google.com/file/d/1D479FLjp9aO3Mg8g6Lpj9oRViWacurA6/view?usp=sharing

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00
...,...,...,...,...,...,...,...,...,...,...,...,...
18363,18363,Shopping,1920,26.150504,91.733531,1517,truck,average,6,0,19-12-2016,14:30:00
18364,18364,Shopping,1920,26.150504,91.733531,1487,car,low,3,0,19-12-2016,15:03:00
18365,18365,Shopping,1920,26.150504,91.733531,1432,cycle,low,3,0,19-12-2016,15:29:00
18366,18366,Shopping,1920,26.150504,91.733531,1321,car,low,2,0,19-12-2016,16:03:00


In [ ]:
# Combine the 'LastUpdatedDate' and 'LastUpdatedTime' columns into a single datetime column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')

# Sort the DataFrame by the new 'Timestamp' column and reset the index
df = df.sort_values('Timestamp').reset_index(drop=True)

In [ ]:
# Save the selected columns to a CSV file for streaming or downstream processing
df[["Timestamp", "Occupancy", "Capacity"]].to_csv("parking_stream.csv", index=False)

# Note: Only three features are used here for simplicity.# Participants are expected to incorporate additional relevant features in their models.

In [ ]:
# Define the schema for the streaming data using Pathway
# This schema specifies the expected structure of each data row in the stream

class ParkingSchema(pw.Schema):
    Timestamp: str   # Timestamp of the observation (should ideally be in ISO format)
    Occupancy: int   # Number of occupied parking spots
    Capacity: int    # Total parking capacity at the location

In [ ]:
# Load the data as a simulated stream using Pathway's replay_csv function
# This replays the CSV data at a controlled input rate to mimic real-time streaming
# input_rate=1000 means approximately 1000 rows per second will be ingested into the stream.

data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=1000)

In [ ]:
# Define the datetime format to parse the 'Timestamp' column
fmt = "%Y-%m-%d %H:%M:%S"

# Add new columns to the data stream:
# - 't' contains the parsed full datetime
# - 'day' extracts the date part and resets the time to midnight (useful for day-level aggregations)
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

# Step 2: Making a simple pricing function

In [ ]:
# Define a daily tumbling window over the data stream using Pathway
# This block performs temporal aggregation and computes a dynamic price for each day
import datetime

delta_window = (
    data_with_time.windowby(
        pw.this.t,  # Event time column to use for windowing (parsed datetime)
        instance=pw.this.day,  # Logical partitioning key: one instance per calendar day
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),  # Fixed-size daily window
        behavior=pw.temporal.exactly_once_behavior()  # Guarantees exactly-once processing semantics
    )
    .reduce(
        t=pw.this._pw_window_end,                        # Assign the end timestamp of each window
        occ_max=pw.reducers.max(pw.this.Occupancy),      # Highest occupancy observed in the window
        occ_min=pw.reducers.min(pw.this.Occupancy),      # Lowest occupancy observed in the window
        cap=pw.reducers.max(pw.this.Capacity),           # Maximum capacity observed (typically constant per spot)
    )
    .with_columns(
        # Compute the price using a simple dynamic pricing formula:
        #
        # Pricing Formula:
        #     price = base_price + demand_fluctuation
        #     where:
        #         base_price = 10 (fixed minimum price)
        #         demand_fluctuation = (occ_max - occ_min) / cap
        #
        # Intuition:
        # - The greater the difference between peak and low occupancy in a day,
        #   the more volatile the demand is, indicating potential scarcity.
        # - Dividing by capacity normalizes the fluctuation (to stay in [0,1] range).
        # - This fluctuation is added to the base price of 10 to set the final price.
        # - Example: If occ_max = 90, occ_min = 30, cap = 100
        #            => price = 10 + (90 - 30)/100 = 10 + 0.6 = 10.6

        price=10 + (pw.this.occ_max - pw.this.occ_min) / pw.this.cap
    )
)

# Step 3: Visualizing Daily Price Fluctuations with a Bokeh Plot

**Note:** The Bokeh plot in the next cell will only be generated after you run the `pw.run()` cell (i.e., the final cell).


In [ ]:
# Activate the Panel extension to enable interactive visualizations
pn.extension()

# Define a custom Bokeh plotting function that takes a data source (from Pathway) and returns a figure
def price_plotter(source):
    # Create a Bokeh figure with datetime x-axis
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Daily Parking Price",
        x_axis_type="datetime",  # Ensure time-based data is properly formatted on the x-axis
    )
    # Plot a line graph showing how the price evolves over time
    fig.line("t", "price", source=source, line_width=2, color="navy")

    # Overlay red circles at each data point for better visibility
    fig.circle("t", "price", source=source, size=6, color="red")

    return fig

# Use Pathway's built-in .plot() method to bind the data stream (delta_window) to the Bokeh plot
# - 'price_plotter' is the rendering function
# - 'sorting_col="t"' ensures the data is plotted in time order
viz = delta_window.plot(price_plotter, sorting_col="t")

# Create a Panel layout and make it servable as a web app
# This line enables the interactive plot to be displayed when the app is served
pn.Column(viz).servable()

Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [ ]:
# Start the Pathway pipeline execution in the background
# - This triggers the real-time data stream processing defined above
# - %%capture --no-display suppresses output in the notebook interface

%%capture --no-display
pw.run()

Output()

##Another way we can perform data analytics

In [ ]:

# 1. Install Required Libraries (Colab Only)
!pip install pathway bokeh

# 2. Imports
import pandas as pd
import numpy as np
import pathway as pw
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.layouts import column
output_notebook()

In [ ]:
# 3. Load Dataset
df = pd.read_csv("/content/dataset.csv")
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format='%d-%m-%Y %H:%M:%S')
df.head()

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime,Timestamp
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00,2016-10-04 07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00,2016-10-04 08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00,2016-10-04 08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00,2016-10-04 09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00,2016-10-04 09:59:00


In [ ]:

# 4. Simulate Real-Time DataStream
def simulate_stream(df, delay=0.5):
    import time
    for _, row in df.iterrows():
        yield row.to_dict()
        time.sleep(delay)

In [ ]:

# 5. Helper Functions for Models

def model_1_baseline(price, occupancy, capacity, alpha=0.1):
    """Model 1: Linear increase based on occupancy"""
    return price + alpha * (occupancy / capacity)

def model_2_demand_based(base_price, occupancy, capacity, queue, traffic, is_special, vehicle_weight,
                         alpha=1.0, beta=0.5, gamma=0.3, delta=0.4, epsilon=0.2):
    """Model 2: Price based on demand score"""
    demand = (
        alpha * (occupancy / capacity) +
        beta * queue -
        gamma * traffic +
        delta * is_special +
        epsilon * vehicle_weight
    )
    normalized_demand = (demand - 0.5) / 2  # scale between -0.25 to +0.25 approx
    price = base_price * (1 + normalized_demand)
    return min(max(price, 0.5 * base_price), 2 * base_price)  # bounds between 0.5x and 2x

In [ ]:

# 6. Optional: Compute Competitor Pricing Influence
from geopy.distance import geodesic

def competitive_adjustment(my_lat, my_lon, my_price, other_lots, threshold=0.5):
    """Model 3: Adjust price using competitor lot info"""
    nearby_prices = []
    for lot in other_lots:
        dist = geodesic((my_lat, my_lon), (lot['Latitude'], lot['Longitude'])).meters
        if dist <= 300:  # within 300m
            nearby_prices.append(lot['CurrentPrice'])
    if not nearby_prices:
        return my_price
    avg_competitor_price = np.mean(nearby_prices)
    if my_price > avg_competitor_price:
        return my_price - threshold  # reduce slightly
    elif my_price < avg_competitor_price:
        return my_price + threshold  # increase slightly
    return my_price

In [ ]:

# 7. Real-Time Pricing Simulation with Visualization

# --- Step 1: Identify Timestamp Column ---

# If your dataset already has a usable timestamp column (like 'LastUpdated' or 'Timestamp'), use it.
# Otherwise, list available columns first
print("Available columns:", df.columns.tolist())

# Try using a complete timestamp if it exists
if 'LastUpdated' in df.columns:
    df['Timestamp'] = pd.to_datetime(df['LastUpdated'], errors='coerce')
elif 'LastUpdatedDate' in df.columns and 'LastUpdatedTime' in df.columns:
    df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], errors='coerce')
elif 'Timestamp' in df.columns:
    df['Timestamp'] = pd.to_datetime(df['Timestamp'], errors='coerce')
else:
    raise KeyError("No valid timestamp fields found. Please check your dataset headers.")


# --- Step 2: Clean & Prepare Required Columns ---

essential_cols = ['SystemCodeNumber', 'Latitude', 'Longitude', 'Occupancy', 'Capacity',
                  'QueueLength', 'VehicleType', 'TrafficConditionNearby', 'IsSpecialDay', 'Timestamp']

df = df[essential_cols].dropna().copy()


# --- Step 3: Group by Timestamp for Efficiency ---

timestamp_groups = dict(tuple(df.groupby('Timestamp')))

# --- Step 4: Initialize Pricing Storage ---

source_dict = {}
lot_prices = {lot_id: 10.0 for lot_id in df['SystemCodeNumber'].unique()}


# --- Step 5: Real-Time Stream Simulation ---

for row in simulate_stream(df, delay=0.01):
    lot_id = row['SystemCodeNumber']
    base_price = 10.0
    prev_price = lot_prices.get(lot_id, base_price)

    # Vehicle type weight
    vt_map = {'car': 1.0, 'bike': 0.5, 'truck': 1.5}
    vt_weight = vt_map.get(str(row['VehicleType']).lower(), 1.0)

    # Traffic level mapping
    traffic_map = {"Low": 1, "Medium": 2, "High": 3}
    traffic_level = traffic_map.get(str(row['TrafficConditionNearby']), 2)

    # Model 2: Demand-Based Price
    price_2 = model_2_demand_based(
        base_price=base_price,
        occupancy=row['Occupancy'],
        capacity=row['Capacity'],
        queue=row['QueueLength'],
        traffic=traffic_level,
        is_special=row['IsSpecialDay'],
        vehicle_weight=vt_weight
    )

    # Model 3: Competitive Adjustment
    current_time = row['Timestamp']
    current_group = timestamp_groups.get(current_time, pd.DataFrame())

    if not current_group.empty and 'SystemCodeNumber' in current_group.columns:
        other_lots_df = current_group[current_group['SystemCodeNumber'] != lot_id].copy()
        other_lots_df['CurrentPrice'] = other_lots_df['SystemCodeNumber'].map(lot_prices)
        other_lots_df = other_lots_df.dropna(subset=['CurrentPrice'])

        price_3 = competitive_adjustment(
            row['Latitude'], row['Longitude'], price_2, other_lots_df.to_dict(orient='records')
        )
    else:
        price_3 = price_2  # fallback

    # Update lot price
    lot_prices[lot_id] = price_3

    # Store for plotting
    if lot_id not in source_dict:
        source_dict[lot_id] = {'timestamps': [], 'prices': []}
    source_dict[lot_id]['timestamps'].append(current_time)
    source_dict[lot_id]['prices'].append(price_3)

Available columns: ['ID', 'SystemCodeNumber', 'Capacity', 'Latitude', 'Longitude', 'Occupancy', 'VehicleType', 'TrafficConditionNearby', 'QueueLength', 'IsSpecialDay', 'LastUpdatedDate', 'LastUpdatedTime', 'Timestamp']


In [ ]:
# 8. Visualize Prices with Bokeh

from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.layouts import column

output_notebook()

plots = []

for lot_id, data in source_dict.items():
    src = ColumnDataSource(data={'x': data['timestamps'], 'y': data['prices']})
    p = figure(title=f"Lot {lot_id} - Real-Time Pricing", x_axis_type='datetime', height=250)
    p.line(x='x', y='y', source=src, line_width=2, legend_label=f"Lot {lot_id}")
    p.yaxis.axis_label = "Price ($)"
    p.xaxis.axis_label = "Time"
    plots.append(p)

show(column(*plots[:5]))  # Show only first 5 plots for brevity


## Report Section

### Assumptions:
- Base price is fixed at $10.
- Occupancy-based pricing grows linearly (Model 1).
- Demand-based pricing uses a weighted linear combination (Model 2).
- Nearby lot prices and distances affect pricing for competitive adjustments (Model 3).

### How price changes:
- Higher occupancy or longer queue increases demand.
- Heavy traffic reduces demand.
- Special days and trucks increase pricing influence.
- Competitor lots nearby are used to increase or decrease the final price.

---